In [75]:
import os
from bs4 import BeautifulSoup
%pip install playwright
!playwright install
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [76]:
SEASONS = list(range(2016, 2025))

In [77]:
DATA_DIR = 'games'
STANDINGS_DIR = os.path.join(DATA_DIR, 'standings')
SCORES_DIR = os.path.join(DATA_DIR, 'scores')


# Ensure directories exist
os.makedirs(STANDINGS_DIR, exist_ok=True)
os.makedirs(SCORES_DIR, exist_ok=True)

print(f"Data will be saved in: {DATA_DIR}")
print(f"Standings will be saved in: {STANDINGS_DIR}")
print(f"Scores will be saved in: {SCORES_DIR}")

Data will be saved in: games
Standings will be saved in: games/standings
Scores will be saved in: games/scores


In [78]:
import time

async def get_html(url, selector, sleep=30, retries=5):
    html = None
    for _ in range(1, retries+1):
        time.sleep(sleep)
        try:
            async with async_playwright() as p:
                browser = await p.chromium.launch()
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:
            print(f'Timeout on attempt {_} for {url}')
            continue
        else:
            break
    return html

In [79]:
async def scrape_seasons(season):
    url = f'https://www.basketball-reference.com/leagues/NBA_{season}_games.html'
    html = await get_html(url, '#content .filter')
    soup = BeautifulSoup(html)
    links = soup.find_all('a')
    href = [link['href'] for link in links]
    standings_pages = [f"https://www.basketball-reference.com{h}" for h in href] 
    
    for url in standings_pages:
        save_path = os.path.join(STANDINGS_DIR, url.split('/')[-1])
        if os.path.exists(save_path):
            print(f'{save_path} already exists')
            continue

        html = await get_html(url, '#all_schedule')
        with open(save_path, 'w+') as f:
            f.write(html)


In [80]:
season = 2010
url = f'https://www.basketball-reference.com/leagues/NBA_{season}_games.html'

In [81]:
standings_files = os.listdir(STANDINGS_DIR)

In [82]:
async def scrape_box_scores(standings_file):
    with open(standings_file, 'r') as f:
        html = f.read()

    soup = BeautifulSoup(html)
    links = soup.find_all('a')
    href = [link.get("href") for link in links]
    box_scores = [h for h in href if h and 'boxscores' in h and '.html' in h]
    box_scores = [f"https://www.basketball-reference.com{h}" for h in box_scores]

    for url in box_scores:
        save_path = os.path.join(SCORES_DIR, url.split('/')[-1])
        if os.path.exists(save_path):
            print(f'{save_path} already exists')
            continue

        html = await get_html(url, '#content')
        if not html:
            print(f'Failed to get {url}')
            continue
        with open(save_path, 'w+') as f:
            f.write(html)

In [83]:
standings_files = [s for s in standings_files if '.html' in s]

for file in standings_files:
    file_path = os.path.join(STANDINGS_DIR, file)
    await scrape_box_scores(file_path)

games/scores/202110190MIL.html already exists
games/scores/202110190LAL.html already exists
games/scores/202110200CHO.html already exists
games/scores/202110200DET.html already exists
games/scores/202110200NYK.html already exists
games/scores/202110200TOR.html already exists
games/scores/202110200MEM.html already exists
games/scores/202110200MIN.html already exists
games/scores/202110200NOP.html already exists
games/scores/202110200SAS.html already exists
games/scores/202110200UTA.html already exists
games/scores/202110200POR.html already exists
games/scores/202110200PHO.html already exists
games/scores/202110210ATL.html already exists
games/scores/202110210MIA.html already exists
games/scores/202110210GSW.html already exists
games/scores/202110220ORL.html already exists
games/scores/202110220WAS.html already exists
games/scores/202110220CLE.html already exists
games/scores/202110220BOS.html already exists
games/scores/202110220PHI.html already exists
games/scores/202110220HOU.html alr

In [84]:
standings_files


['NBA_2022_games-october.html',
 'NBA_2021_games-june.html',
 'NBA_2024_games-may.html',
 'NBA_2020_games-march.html',
 'NBA_2020_games-september.html',
 'NBA_2020_games-january.html',
 'NBA_2020_games-august.html',
 'NBA_2023_games-may.html',
 'NBA_2019_games-april.html',
 'NBA_2023_games-april.html',
 'NBA_2022_games-may.html',
 'NBA_2019_games-february.html',
 'NBA_2018_games-february.html',
 'NBA_2023_games-january.html',
 'NBA_2016_games-april.html',
 'NBA_2024_games-march.html',
 'NBA_2021_games-march.html',
 'NBA_2018_games-january.html',
 'NBA_2024_games-november.html',
 'NBA_2017_games-february.html',
 'NBA_2024_games-december.html',
 'NBA_2016_games-february.html',
 'NBA_2017_games-october.html',
 'NBA_2018_games-april.html',
 'NBA_2020_games-december.html',
 'NBA_2019_games-october.html',
 'NBA_2020_games-november.html',
 'NBA_2021_games-may.html',
 'NBA_2021_games-december.html',
 'NBA_2022_games-april.html',
 'NBA_2022_games-december.html',
 'NBA_2023_games-november.html',